In [8]:
#!/usr/bin/env python

import torch, time

from torch import Tensor

######################################################################
print('# 1 #')

m = Tensor(13, 13).fill_(1)

m.narrow(0, 1, 1).fill_(2)
m.narrow(1, 1, 1).fill_(2)
m.narrow(0, 6, 1).fill_(2)
m.narrow(1, 6, 1).fill_(2)
m.narrow(0, 11, 1).fill_(2)
m.narrow(1, 11, 1).fill_(2)

m.narrow(0, 3, 2).narrow(1, 3, 2).fill_(3)
m.narrow(0, 3, 2).narrow(1, 8, 2).fill_(3)
m.narrow(0, 8, 2).narrow(1, 3, 2).fill_(3)
m.narrow(0, 8, 2).narrow(1, 8, 2).fill_(3)

print(m)

######################################################################
# numpy style

m = Tensor(13, 13).fill_(1)

m[3:5,3:5] = 3
m[8:10,3:5] = 3
m[3:5,8:10] = 3
m[8:10,8:10] = 3

m[:,1::5] = 2
m[1::5,:] = 2

print(m)

######################################################################
print('# 2 #')

m = Tensor(20, 20).normal_()
d = torch.diag(torch.arange(1, m.size(0)+1))
d = d.float()
q = m.mm(d).mm(m.inverse())
v, _ = q.eig()
print('Eigenvalues', v.narrow(1, 0, 1).squeeze().sort()[0])

######################################################################
print('# 3 #')

d = 5000
a = Tensor(d, d).normal_()
b = Tensor(d, d).normal_()

time1 = time.perf_counter()
c = torch.mm(a, b)
time2 = time.perf_counter()

print('Throughput {:e} flop/s'.format((d * d * d)/(time2 - time1)))

######################################################################
print('# 4 #')

def mul_row(m):
    r = torch.Tensor(m.size())
    for i in range(0, m.size(0)):
        for j in range(0, m.size(1)):
            r[i, j] = m[i, j] * (i+1)
    return r

def mul_row_fast(m):
    d = m.size(0)
    c = torch.arange(1, d + 1).view(d, 1).expand_as(m)

    return m.mul(c)

m = Tensor(10000, 400).normal_(5.0)

time1 = time.perf_counter()
a = mul_row(m)
time2 = time.perf_counter()
b = mul_row_fast(m)
time3 = time.perf_counter()

print('Speed ratio', (time2 - time1) / (time3 - time2))

print('Sanity check: error is ', torch.norm(a - b))

######################################################################

# 1 #
tensor([[1., 2., 1., 1., 1., 1., 2., 1., 1., 1., 1., 2., 1.],
        [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [1., 2., 1., 1., 1., 1., 2., 1., 1., 1., 1., 2., 1.],
        [1., 2., 1., 3., 3., 1., 2., 1., 3., 3., 1., 2., 1.],
        [1., 2., 1., 3., 3., 1., 2., 1., 3., 3., 1., 2., 1.],
        [1., 2., 1., 1., 1., 1., 2., 1., 1., 1., 1., 2., 1.],
        [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [1., 2., 1., 1., 1., 1., 2., 1., 1., 1., 1., 2., 1.],
        [1., 2., 1., 3., 3., 1., 2., 1., 3., 3., 1., 2., 1.],
        [1., 2., 1., 3., 3., 1., 2., 1., 3., 3., 1., 2., 1.],
        [1., 2., 1., 1., 1., 1., 2., 1., 1., 1., 1., 2., 1.],
        [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [1., 2., 1., 1., 1., 1., 2., 1., 1., 1., 1., 2., 1.]])
tensor([[1., 2., 1., 1., 1., 1., 2., 1., 1., 1., 1., 2., 1.],
        [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [1., 2., 1., 1., 1., 1., 2., 1., 1., 1., 1., 2., 1.],
 

RuntimeError: Lapack Error in geev : Illegal Argument 2 at /Users/soumith/code/builder/wheel/pytorch-src/aten/src/TH/generic/THTensorLapack.cpp:335

In [1]:
import torch

In [2]:
d = 20
m = torch.Tensor(d,d).normal_()
m_1 = torch.inverse(m)
diag = torch.diag(torch.arange(1,21))
diag = diag.float()
mul = m_1.mm(diag.mm(m))
eig = torch.eig(mul)



Generate two square matrices of dimension 5000 × 5000 filled with random Gaussian coefficients, compute their product, measure the time it takes, and estimate how many floating point products have been executed per second (should be in the billions or tens of billions).

Use torch.normal , torch.mm , and time.perf counter .

In [13]:
import time
d = 5000
m1 = torch.Tensor(d, d).normal_()
m2 = torch.Tensor(d, d).normal_()
t1 = time.perf_counter()
mul = m1.mm(m2)
t2 = time.perf_counter()
print(f"Time is {t2-t1}")
print((d*d*d)/(t2-t1))

Time is 1.5133522740215994
82598085155.56235


In [14]:
print('# 3 #')

d = 5000
a = Tensor(d, d).normal_()
b = Tensor(d, d).normal_()

time1 = time.perf_counter()
c = torch.mm(a, b)
time2 = time.perf_counter()

print('Throughput {:e} flop/s'.format((d * d * d)/(time2 - time1)))
print(time2- time1)

# 3 #
Throughput 8.369771e+10 flop/s
1.4934698089782614


# Playing with strides

Write a function mul row , using python loops, that gets a 2d tensor as argument, and returns a tensor of same size, equal to the one given as argument, with the first row kept unchanged, the second multiplied by two, the third by three, etc. 

Then, write a second version named mul row fast , using tensor operations.
Apply both versions to a matrix of size 10, 000 × 400 and measure the time each takes (there should
be more than two orders of magnitude difference).
Hint: Use torch.arange , torch.view , torch.expand as , torch.mul , and time.perf counter .

In [20]:
def mul_row(tensor):
    r = torch.Tensor(m.size())
    for i in range(len(tensor)):
        r[i,:] = (i+1)*tensor[i,:]
    return r

In [21]:
m = torch.Tensor(4,8).fill_(2.0)
mul_row(m)

tensor([[2., 2., 2., 2., 2., 2., 2., 2.],
        [4., 4., 4., 4., 4., 4., 4., 4.],
        [6., 6., 6., 6., 6., 6., 6., 6.],
        [8., 8., 8., 8., 8., 8., 8., 8.]])